Before starting we need to setup some configuration, like which AI backend to use.

When using the kernel for AI requests, the kernel needs some settings like URL and
credentials to the AI models. The SDK currently supports OpenAI and Azure OpenAI,
other services will be added over time. If you need an Azure OpenAI key, go
[here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=rest-api).

The following code will ask a few questions and save the settings to a local
`settings.json` configuration file, under the [config](config) folder. You can
also edit the file manually if you prefer. **Please keep the file safe.**

### Step 1 Import configuration packages and classes

In order to build the settings file, we need a couple helpers.

In [ ]:
//Install Nuget Packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4"

//Load support classes
#!import config/Settings.cs
#!import config/OpenAIConfiguration.cs
#!import config/OobaboogaConnectorConfiguration.cs

// Creating the Open AI configuration object
var openAIConfig = new OpenAIConfiguration();

// Creating the Oobabooga AI configuration object
var oobaboogaConfig = new OobaboogaConnectorConfiguration();

### Step 2 Define your OpenAI configuration

If you plan on using ChatGPT as a primary connector, enter your OpenAI credentials.
If you only plan on using Oobabooga, you can skip this step.

Run the following code. If you need to find the value and copy and paste, you can
re-run the code and continue from where you left off.

In [12]:

// Asking for the Open AI API key

var openAIKey = await InteractiveKernel.GetInputAsync("Please enter your OpenAI API key");
openAIConfig.ApiKey = openAIKey;


### Step 3 Define your primary Oobabooga configuration

To test Ooobabooga completion providers, or to use Oobabooga as a primary connector for multi-connector tests, define your Oobabooga endpoint settings:

Run the following code. If you need to find the value and copy and paste, you can
re-run the code and continue from where you left off.

In [14]:
var oobaboogaEndpoint = await InteractiveKernel.GetInputAsync("Please enter your Oobabooga endpoint");
oobaboogaConfig.EndPoint = oobaboogaEndpoint;

var oobaboogaBlockingPort = await InteractiveKernel.GetInputAsync("Please enter your Oobabooga API Blocking port (default 5000)");
oobaboogaConfig.BlockingPort = int.Parse(oobaboogaBlockingPort);

var oobaboogaStreamingPort = await InteractiveKernel.GetInputAsync("Please enter your Oobabooga API Streaming port (default 5005)");
oobaboogaConfig.StreamingPort = int.Parse(oobaboogaStreamingPort);



### Step 4: Save Configuration to `settings.json`

In [ ]:
using System.Text.Json;
using System.IO;

// Create a dictionary to hold all configurations
Dictionary<string, object> allConfigs = new Dictionary<string, object>
{
    { "OpenAI", openAIConfig },
    { "Oobabooga", oobaboogaConfig }
};

// Serialize the dictionary to JSON
string jsonString = JsonSerializer.Serialize(allConfigs, new JsonSerializerOptions { WriteIndented = true });

// Save the JSON string to settings.json
File.WriteAllText("config/settings.json", jsonString);

Console.WriteLine("Settings saved to config/settings.json");


If the code above doesn't show any error, you're good to go and run the other notebooks.

### Resetting the configuration

If you want to reset the configuration and start again, please uncomment and run the code below.
You can also edit the [config/settings.json](config/settings.json) manually if you prefer.

In [ ]:
#!import config/Settings.cs

// Uncomment this line to reset your settings and delete the file from disk.
// Settings.Reset();

Now that your environment is all set up, let's dive into
[how to do basic loading of the Semantic Kernel](01-basic-loading-the-kernel.ipynb).